In [1]:
### Testing for updateTrashCanStatus
from CompareImagesModel import *

model = CompareImagesModel()

c:\Users\krome\OneDrive\Desktop\innovaite\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\krome\OneDrive\Desktop\innovaite\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
import numpy as np
import pandas as pd
import os

def load_image_paths_from_folders_to_df(base_path):
    '''
    Provide base path for images and will return a pandas dataframe with the image paths and labels.
    '''
    image_paths = []
    labels = []
    label_names = os.listdir(base_path)

    for label_idx, label_name in enumerate(label_names):
        label_folder = os.path.join(base_path, label_name)

        if os.path.isdir(label_folder):
            for image_name in os.listdir(label_folder):
                image_path = os.path.join(label_folder, image_name)

                if image_path.endswith(('.png', '.jpg', '.jpeg')):  # Filter by image extensions
                    image_paths.append(image_path)
                    labels.append(label_name)  # Label is the name of the folder in label_names

    df = pd.DataFrame({'IMAGEPATH': image_paths, 'ID': labels })

    return df

In [3]:
Cleaned = "../../Trash Cans Cleaned"  # Replace with your destination path
trashcan_images_df = load_image_paths_from_folders_to_df(Cleaned)
trashcan_images_df['ID'] = trashcan_images_df['ID'].astype(int)

In [4]:
trashcan_images_df["ID"].unique()

array([ 1, 17, 18, 19,  2, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
       32, 33, 38, 39,  4,  5,  6,  7])

In [5]:
from PIL import Image
# Load the baseline image and extract its features
trashcan_id = 21
# get all image paths for trashcan of trashcan_id
filtered_paths = trashcan_images_df[trashcan_images_df['ID'] == trashcan_id]["IMAGEPATH"].values
print(type(filtered_paths))

baseline_img = Image.open(filtered_paths[0])
other_img = Image.open(filtered_paths[1])

print(model.predict(baseline_img, other_img))

<class 'numpy.ndarray'>
0.84914136


In [9]:
from updateTrashCanStatus import matchTrashCanID
trash_can_ids = [19, 20, 21, 22]
image_folder_paths = []
for id in trash_can_ids:
    paths = trashcan_images_df[trashcan_images_df['ID'] == id]["IMAGEPATH"].values
    image_folder_paths.append(os.path.dirname(paths[0]))

trashcan_id = 20
filtered_paths = trashcan_images_df[trashcan_images_df['ID'] == trashcan_id]["IMAGEPATH"].values
image = Image.open(filtered_paths[0])

pred_id = matchTrashCanID(image, image_folder_paths, trash_can_ids, model)

#print(trash_can_ids)
#print(image_folder_paths)
print("Expected:", trashcan_id, "Returned:", pred_id)

Expected: 20 Returned: 20


In [10]:
! pip install sqlite3

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [14]:
import sqlite3

# Create (or connect to) the database
conn = sqlite3.connect("database.db")
cursor = conn.cursor()

# Read the .sql file
with open("trash_can.sql", "r") as f:
    sql_script = f.read()

# Execute the SQL script
cursor.executescript(sql_script)

# Commit and close
conn.commit()
conn.close()


In [5]:
from updateTrashCanStatus import updateTrashCanStatus
DATABASE_PATH = "database.db"
import sqlite3

# Call function after defining config
trash_can_id = 19
updateTrashCanStatus(trash_can_id, 1)

# Connect to database and verify result
try:
    connection = sqlite3.connect(DATABASE_PATH)
    cursor = connection.cursor()

    # Query the updated row
    cursor.execute(f"SELECT * FROM trash_cans WHERE trash_can_id = {trash_can_id};")
    result = cursor.fetchone()

    print("\nDatabase Output:")
    print(result)  # This should print the updated trash can row

except Exception as e:
    print(f"Database Error: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()

Marking trash can 19 as FULL.

Database Output:
(19, 42.338979, -71.0885834, 1, '2025-02-16 07:21:28')


In [ ]:
from updateTrashCanStatus import getNearbyTrashCanImages
longitude, latitude = 42, -71
image_folder_paths, trash_can_ids = getNearbyTrashCanImages(longitude, latitude, range=1)